## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [28]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations_ordered', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [29]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,0.024611,-0.295978,0.102216,0.077605
1,0.024394,-0.310827,0.111588,0.066557
2,0.029112,-0.284298,0.101918,0.049235
3,0.074158,-0.274682,0.064711,0.008161
4,0.068909,-0.275075,0.072048,0.005100


In [30]:
Eig.shape

(65, 4)

In [64]:
Eig['station']=Res_eig['stations_ordered']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00041018', u'USC00260507', u'USC00044773', u'USC00043491', u'USC00042334']


,0,1,2,3
station,,,,
US1CANV0001,0.024611,-0.295978,0.102216,0.077605
US1CANV0004,0.024394,-0.310827,0.111588,0.066557
US1CAPC0001,0.029112,-0.284298,0.101918,0.049235
US1NVWH0003,0.074158,-0.274682,0.064711,0.008161
US1NVWH0019,0.068909,-0.275075,0.072048,0.005100


In [65]:
table=Eig.join(Stations,how='left')

In [66]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
US1CANV0001,NEVADA CITY 3.4 NNW,39.3067,-121.0399,728.5,0.024611,-0.295978,0.102216,0.077605
US1CANV0004,GRASS VALLEY 5.5 ESE,39.1900,-120.9631,842.2,0.024394,-0.310827,0.111588,0.066557
US1CAPC0001,SODA SPRINGS 1.5 SSW,39.3021,-120.3837,2099.2,0.029112,-0.284298,0.101918,0.049235
US1NVWH0003,RENO 10.0 S,39.3954,-119.7900,1554.5,0.074158,-0.274682,0.064711,0.008161


In [81]:
#table.sort(columns=[1], ascending=True)
table
#group0= table.sort(columns=[1], ascending=True).head(10)
group0 = table.head(8)
group1 = table.sort(columns=[0], ascending=False).head(9)
group2 = table.sort(columns=[1], ascending=True).head(13)

c:\Users\wenyan\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
c:\Users\wenyan\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [75]:
group0

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
US1CANV0001,NEVADA CITY 3.4 NNW,39.3067,-121.0399,728.5,0.024611,-0.295978,0.102216,0.077605
US1CANV0004,GRASS VALLEY 5.5 ESE,39.1900,-120.9631,842.2,0.024394,-0.310827,0.111588,0.066557
US1CAPC0001,SODA SPRINGS 1.5 SSW,39.3021,-120.3837,2099.2,0.029112,-0.284298,0.101918,0.049235
US1NVWH0003,RENO 10.0 S,39.3954,-119.7900,1554.5,0.074158,-0.274682,0.064711,0.008161
US1NVWH0019,RENO 9.0 W,39.5184,-119.9935,1493.5,0.068909,-0.275075,0.072048,0.005100
US1NVWH0025,RENO 3.9 W,39.5420,-119.8958,1570.0,0.079798,-0.294577,0.065570,0.012297
US1NVWH0035,RENO 4.5 S,39.4736,-119.8286,1486.8,0.078467,-0.277221,0.057284,0.009583
US1NVWH0059,RENO 8.0 S,39.4242,-119.8000,1478.0,0.073823,-0.302623,0.071452,0.023963


In [76]:
group1

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
US1NVWH0025,RENO 3.9 W,39.5420,-119.8958,1570.0,0.079798,-0.294577,0.065570,0.012297
US1NVWH0035,RENO 4.5 S,39.4736,-119.8286,1486.8,0.078467,-0.277221,0.057284,0.009583
US1NVWH0003,RENO 10.0 S,39.3954,-119.7900,1554.5,0.074158,-0.274682,0.064711,0.008161
US1NVWH0059,RENO 8.0 S,39.4242,-119.8000,1478.0,0.073823,-0.302623,0.071452,0.023963
US1NVWH0019,RENO 9.0 W,39.5184,-119.9935,1493.5,0.068909,-0.275075,0.072048,0.005100
USC00049040,TRUCKEE-TAHOE AP DIST,39.3181,-120.1450,1798.3,0.046280,-0.276433,0.095704,0.022691
US1CAPC0001,SODA SPRINGS 1.5 SSW,39.3021,-120.3837,2099.2,0.029112,-0.284298,0.101918,0.049235
US1CANV0001,NEVADA CITY 3.4 NNW,39.3067,-121.0399,728.5,0.024611,-0.295978,0.102216,0.077605
US1CANV0004,GRASS VALLEY 5.5 ESE,39.1900,-120.9631,842.2,0.024394,-0.310827,0.111588,0.066557


In [70]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors = ['#FF0000', '#00FF00', '#0000FF', '#000000']

## Map

In [71]:
scope_max=table.max()
scope_min=table.min()
min_lat,max_lat,min_long,max_long = box = (scope_min['latitude'], scope_max['latitude'], scope_min['longitude'], scope_max['longitude'])

In [90]:
type(group1.index.values)

numpy.ndarray

In [83]:
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 7
lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

k=0
for index,row in group1.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    #if k >=3: break
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
    k+=1

m    

In [124]:
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 7
lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

k = 0
for index,row in table.reset_index().iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly

k=0
for index,row in group1.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    color = '#FF0' if k<5 else '#0FF'
    c = Circle(location=(_lat,_long), radius=int(5000), weight=4,
               color=color, opacity=0.8, fill_opacity=.0)
    m += c
    k+=1

m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.